---
# 무관심 가사 20개 Markov Chain 하기

In [1]:
import pandas as pd

In [7]:
ballad = pd.read_csv("../../Data/train_data.csv")

In [10]:
ballad[ballad.감정 == "regret"].drop(index=range(40,60))

,Unnamed: 0,가수명,노래제목,발매일,가사,좋아요수,감정


In [12]:
ballad.reset_index(drop=True)

,Unnamed: 0,가수명,노래제목,발매일,가사,좋아요수,감정
0,0,양다일,Darling,2020-11-23,Oh darling 처음 널 만나면 수줍은 대화들로 사랑에 빠질 거야 My darl...,10907,happy
1,1,장범준,잠이 오질 않네요,2020-10-24,당신은 날 설레게 만들어 조용한 내 마음 자꾸만 춤추게 해 얼마나 얼마나 날 떨리게...,159018,happy
2,2,백아연,어디라도,2020-11-18,우연의 다른 이름은 운명이란 걸 배웠죠 우리 사랑의 끝이 없는 걸 이제서야 알았죠 ...,1622,happy
3,3,김제훈,Love Again,2017-02-06,언제부터였을까 따뜻한 휴식을 찾고 있을 때 찾고 있을 때 그때부터였을까 맘이 포근해...,980,happy
4,4,김제훈,잠잠,2020-03-17,오랜만에 거길 가봤어 아무도 없는 둘만이 알던 곳 함께 걷던 거리에 서서 아무 말도...,548,happy
...,...,...,...,...,...,...,...
75,75,#안녕,너의 번호를 누르고 (Prod. 영화처럼),2019-11-30,우연히 너를 만나서 너의 옆자리에 앉아 그렇게 우린 친해졌어 짧은 시간에 그렇게 가...,109930,indifference
76,76,부활,Never Ending Story,2002-09-01,손 닿을 수 없는 저기 어딘가 오늘도 난 숨쉬고 있지만 너와 머물던 작은 의자 위에...,70543,indifference
77,77,Chocolat.D (쇼콜라디),그날에 우리,2016-02-05,그날에 우리 예감했었지 끊어질 것 같은 우리의 사랑 차마 말하지 못했지 마지막 그 ...,61,indifference
78,78,정승수,되돌릴 수 없는 이별을 해,2021-01-10,혼자 집에 걸어가는 길이 어색해 우리 예전처럼 함께 걷던 그 날이 무색해질 만큼 변...,878,indifference


In [13]:
ballad_lyrics_indifference = ballad[ballad.감정 == 'indifference'].가사

In [14]:
ballad_lyrics_indifference= pd.DataFrame(ballad_lyrics_indifference)
ballad_lyrics_indifference.to_csv("../../Data/ballad_lyrics_indifference.csv",index=False)

In [15]:
import markov
from konlpy.tag import Okt
import json

In [16]:
ballad_file = "../../Note/Markov/ballad_indifference.txt"
dict_file = "../../Note/Markov/markov_ballad_indifference.json"

In [17]:
file = open("../../Data/ballad_lyrics_indifference.csv",encoding='UTF-8')
text = str(file.readlines())
text

"['가사\\n', '반복된 하루 사는 일에 지칠 때면 내게 말해요 항상 그대의 지쳐있는 마음에 조그만 위로 돼줄게요 요즘 유행하는 영화 보고플 땐 내게 말해요 내겐 그대의 작은 부탁조차도 조그만 행복이죠 아무런 약속도 없는 일요일 오후 늦게 잠에서 깨 이유 없이 괜히 서글퍼질 땐 그대 곁엔 세상 누구보다 그댈 이해하는 내 자신보다 그댈 먼저 생각하는 사람이 있죠 오랫동안 항상 지켜왔죠 그대 빈자리 이젠 들어와 편히 쉬어요  혼자서 밥 먹기 싫을 땐 다른 사람 찾지 말아요 내겐 그대의 짜증 섞인 투정도 조그만 기쁨이죠 아무런 약속도 없는 일요일 오후 누군가 만나서 하루 종일 거릴 걷고 싶을 땐 그대 곁엔 세상 누구보다 그댈 이해하는 내 자신보다 그댈 먼저 생각하는 사람이 있죠  오랫동안 항상 지켜왔죠 그대 빈자리 이젠 들어와 편히 쉬어요\\n', '나의 그늘이 좋다고 하는 너에게 무슨 말을 해야 좋을까 이 순간에도 가난한 나의 마음인데도 넌 왜 웃어 보일까 미운 말을 쏟아내 상처 주긴 쉬운데 넌 왜 아직 거기 서 있는데 해는 뜨지 않는데 내게 기댄 너는 나의 그늘이 좋대 어둠이 내리고 모두 떠나가도 가깝지 않은 곳 어디에서 너는 나의 믿음이 되어 미운 말을 쏟아내 상처 주긴 쉬운데 넌 왜 아직 거기 서 있는데 해는 뜨지 않는데 내게 기댄 너는 나의 그늘이 좋대 아 나의 그늘이 나의 그늘이 아 나의 그늘이 나의 그늘이 \\n', 'Moonlight 나를 감싸안아 데려다줘 그럼 너의 온기를 간직하겠어 내가 어디에 숨어있든 다시 날 찾아줘 보기좋게 비껴나온 날 너의 빛으로 비춰줘 내 모든 발자욱과 내 창문 밖에 남은 너와 나의 흔적들 내 모든 것들이 다 그대로 남아 다시 날 채우네 Moonlight 내가 힘이 들 때 찾아와줘 그럼 네 빛을 따라 걸어가겠어 내가 어디에 숨어있든 다시 날 찾아줘 보기좋게 비껴나온 날 너의 빛으로 비춰줘 내 모든 발자욱과 내 창문 밖에 남은 너와 나의 흔적들 내 모든 것들이 다 그대로 남아 다시 날 채우네 \\n', '잊는다는 게 참 그

In [18]:
text = text.split(",")[1:]

In [19]:
total = ""
for i in range(len(text)):
     total += " " + text[i] + "." 

In [20]:
len(total)

9009

In [21]:
total= total.rstrip("].")

In [22]:
total += "."

In [23]:
total

"  '반복된 하루 사는 일에 지칠 때면 내게 말해요 항상 그대의 지쳐있는 마음에 조그만 위로 돼줄게요 요즘 유행하는 영화 보고플 땐 내게 말해요 내겐 그대의 작은 부탁조차도 조그만 행복이죠 아무런 약속도 없는 일요일 오후 늦게 잠에서 깨 이유 없이 괜히 서글퍼질 땐 그대 곁엔 세상 누구보다 그댈 이해하는 내 자신보다 그댈 먼저 생각하는 사람이 있죠 오랫동안 항상 지켜왔죠 그대 빈자리 이젠 들어와 편히 쉬어요  혼자서 밥 먹기 싫을 땐 다른 사람 찾지 말아요 내겐 그대의 짜증 섞인 투정도 조그만 기쁨이죠 아무런 약속도 없는 일요일 오후 누군가 만나서 하루 종일 거릴 걷고 싶을 땐 그대 곁엔 세상 누구보다 그댈 이해하는 내 자신보다 그댈 먼저 생각하는 사람이 있죠  오랫동안 항상 지켜왔죠 그대 빈자리 이젠 들어와 편히 쉬어요\\n'.  '나의 그늘이 좋다고 하는 너에게 무슨 말을 해야 좋을까 이 순간에도 가난한 나의 마음인데도 넌 왜 웃어 보일까 미운 말을 쏟아내 상처 주긴 쉬운데 넌 왜 아직 거기 서 있는데 해는 뜨지 않는데 내게 기댄 너는 나의 그늘이 좋대 어둠이 내리고 모두 떠나가도 가깝지 않은 곳 어디에서 너는 나의 믿음이 되어 미운 말을 쏟아내 상처 주긴 쉬운데 넌 왜 아직 거기 서 있는데 해는 뜨지 않는데 내게 기댄 너는 나의 그늘이 좋대 아 나의 그늘이 나의 그늘이 아 나의 그늘이 나의 그늘이 \\n'.  'Moonlight 나를 감싸안아 데려다줘 그럼 너의 온기를 간직하겠어 내가 어디에 숨어있든 다시 날 찾아줘 보기좋게 비껴나온 날 너의 빛으로 비춰줘 내 모든 발자욱과 내 창문 밖에 남은 너와 나의 흔적들 내 모든 것들이 다 그대로 남아 다시 날 채우네 Moonlight 내가 힘이 들 때 찾아와줘 그럼 네 빛을 따라 걸어가겠어 내가 어디에 숨어있든 다시 날 찾아줘 보기좋게 비껴나온 날 너의 빛으로 비춰줘 내 모든 발자욱과 내 창문 밖에 남은 너와 나의 흔적들 내 모든 것들이 다 그대로 남아 다시 날 채우네 \\n'.  '잊는다는 게 참 그래요 사랑

In [24]:
# 텍스트를 한 줄씩 처리하기
twitter = Okt()
lines = total.split("\n")
for line in lines:
    malist = twitter.pos(line,norm=True,stem=False)
    words = []
    for word in malist:
        if not word[1] in ['Alpha','Punctuation']: # .(마침표)제외한 ? "" ! 를 사용하지 않음
            words.append(word[0])
        if word[0] == ". ":
            words.append(word[0])

In [25]:
# 딕셔너리 생성
dic = markov.make_dic(words)
json.dump(dic,open(dict_file,"w",encoding="utf-8"))

In [26]:
dic

{'@': {'반복': {'된': 1}},
 '반복': {'된': {'하루': 1}},
 '된': {'하루': {'사는': 1, '만큼': 1}, '거': {'죠': 1}},
 '하루': {'사는': {'일': 1},
  '종일': {'거릴': 1, '이별': 1},
  '만': {'더': 4},
  '만큼': {'멀어짐은': 1}},
 '사는': {'일': {'에': 1}},
 '일': {'에': {'지칠': 1},
  '처럼': {'추억': 1},
  '인데': {'왜': 2},
  '거야': {'그대': 1, '그랬잖아요': 1},
  '없이': {'살다보면': 1},
  '들': {'이': 4},
  '없듯': {'우리': 3}},
 '에': {'지칠': {'때': 1},
  '조그만': {'위로': 1},
  '숨어있든': {'다시': 2},
  '묻힌': {'날': 1},
  '나': {'의': 1, '상처': 1},
  '와있지': {'창': 1},
  '괜히': {'생각': 2},
  '조금씩': {'들려오는': 3},
  '누워': {'창': 1},
  '더': {'잠긴': 1},
  '돌아가는': {'길': 1},
  '서서': {'열쇠': 1, '생각': 1},
  '난': {'외톨이': 1},
  '누군가': {'다녀간': 1},
  '얼굴': {'을': 1},
  '감춘': {'두': 1},
  '갇혀': {'방향': 3},
  '울리던': {'우리': 3},
  '거울': {'을': 1},
  '남은': {'건너': 1, '너': 2},
  '비친': {'나': 1},
  '눈': {'을': 2},
  '돌아와': {'돌아와': 2},
  '빠지게': {'되는': 1},
  '만들기': {'로': 1},
  '그': {'어떤': 1},
  '걸음': {'을': 1},
  '가까워질수록': {'너': 1},
  '우리': {'바다': 2, '예감': 1, '이별': 3, '말': 1, '혼자': 1},
  '앉아': {'그렇게': 1},

In [28]:
import os
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
import urllib.request
import os, re, json, random

# 마르코프 체인 딕셔너리 만들기 --- (※1)
def make_dic(words):
    tmp = ["@"]
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 3: continue
        if len(tmp) > 3: tmp = tmp[1:]
        set_word3(dic, tmp)
        if word == ".":
            tmp = ["@"]
            continue
    return dic
# 딕셔너리에 데이터 등록하기 --- (※2)
def set_word3(dic, s3):
    w1, w2, w3 = s3
    if not w1 in dic: dic[w1] = {}
    if not w2 in dic[w1]: dic[w1][w2] = {}
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0
    dic[w1][w2][w3] += 1
# 문장 만들기 --- (※3)
def make_sentence(dic):
    ret = []
    if not "@" in dic: return "no dic" 
    top = dic["@"]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        try:
            w3 = word_choice(dic[w1][w2])
            ret.append(w3)
        except:
            continue
        # if w3 == ".": break
        if len(ret) >= 200: break
        w1, w2 = w2, w3
    ret = "".join(ret)
    # 띄어쓰기
    params = urllib.parse.urlencode({
        "_callback": "",
        "q": ret
    })
    # 네이버 맞춤법 검사기를 사용합니다.
    # data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy?" + params)
    data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn?" + params)
    data = data.read().decode("utf-8")[1:-2]
    data = json.loads(data)
    data = data["message"]["result"]["html"]
    data = soup = BeautifulSoup(data, "html.parser").getText()
    # 리턴
    return data

def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))
# 문장 읽어 들이기 --- (※4)
toji_file = "../../Note/Markov/ballad_indifference.txt"
dict_file = "../../Note/Markov/markov_ballad_indifference.json"
if not os.path.exists(dict_file):
    # 텍스트 파일 읽어 들이기
    with open(toji_file, 'r', encoding='utf-8') as f:
        text = str(f.readlines())

    text = text.replace("…", "") # 현재 koNLPy가 …을 구두점으로 잡지 못하는 문제 임시 해결
    text = text.replace("n", "")
    # 형태소 분석
   # twitter = Twitter()
    twitter=Twitter()
    malist = twitter.pos(text, norm=True)
    words = []
    for word in malist:
        # 구두점 등은 대상에서 제외(단 마침표는 포함)
        if not word[1] in ["Alpha","Punctuation"]:
            words.append(word[0])
        if word[0] == ".":
            words.append(word[0])
    # 딕셔너리 생성
    dic = make_dic(words)
    json.dump(dic, open(dict_file,"w", encoding="utf-8"))
else:
    dic = json.load(open(dict_file,"r"))
# 문장 만들기 — (※6)
for i in range(5):
    s = make_sentence(dic)
    print(s)
    print("—")

반복된 하루만큼 멀어짐은 어쩔 수는 없는 건가요 아주 멀리 있어도 변하는 건 없을 거야 우리 서로 향해 있다면 언제까지나 함께 일 거야 그랬잖아요 그랬었잖아요 그대 내게 그대가 행복하게 지낼 먼 훗날의 모습나 내 마음을 하늘만은 알기를 일부러 몇 발자국 물러나 내가 어떻게 너를 놓아주려 해 근데 그게 맘처럼 쉽게 되지가 않아서 어디서부터 놓아줘야 우리 아름다운 이별을 했고 무뎌진 맘이 센 견딜 수가 없을 만큼 아파서 되돌릴 수 없는 나의 마음인데도 넌 왜 아직 거기 서 있는데 해는 뜨지 않는데 내게 기댄 너는 내게 다가올 것 같아내가 어디에 숨어 있든 다시 날 찾아줘 보기 좋게 비껴 나온 날 너의 행복 빌어줄 그런 드라마 같은 그런 속 깊은 사랑 이 갈수록 익숙해져 편하게 생각했나 봐 날 떠나 가벼워진 사랑은 외려 슬픈 마지막을 가져온다는 걸 그대여 안녕 다시 돌아오지는 마 같은 상처를 받기 너무 두려우니까 널 여전히 사랑해 가끔 널 생각해 아직 모든 게 변했지 숲으로 걷는 다 보고 싶단 말 대신 천천히 걷는다 기억의 너를 이후에 우리 예감했었지 끊어질 것 같은
—
반복된 하루만큼 멀어짐은 어쩔 수는 없는 건가요 아주 멀리 있어도 변하는 건 없을 거야 우리 서로 향해있다면 언제까지나 함께 일 거야 그랬잖아요 그랬었잖아요 그대 내게 그대가 행복하게 지낼 먼 훗날의 모습나 내 마음을 알아 멀어지는 너의 말이 내가 힘이 들 때 찾아와 줘 그럼 네 빛을 따라 걸어가겠어내가 노력해도 변하지 않을 걸 알아 멀어지는 너의 손을 내가 힘이 들 때 찾아와 줘 그럼 네 빛을 따라 걸어가겠어내가 알던 네 모습이 낯설게 느껴져 하루만 더 또 하루만 더 또 하루만 더 혹시나 한번 오늘 어쩌면 아무 말도 안 되는 그이 별 핑계에나 상처 입었죠 사랑이란 모든 것들이 이뤄져가기를 힘겨워한 날의 너를 지킬 수 없었던 아름다운 시절 속에 감춘 두 손이 시리네 어제보다 찬 바람이 지나 네 너는 나의 그늘이 좋다고 하는 너에게 슬픈 안녕을 말해야겠어 너를 지킬 수 없었던 아름다운 시절 속의 스쳐